# Rendu final

Stephen PIET et Théophile LE CLERC

Pour continuer ce projet, nous avons décidé de réutiliser les notations que vous avez prises dans le rendu intermédiaire pour avoir des conventions communes.

## 2 Etude et résolution numérique

1.On rappelle donc le problème auquel nous nous sommes ramené dans le cas d'un seul véhicule:

$f(w) = p^Tw$

$c_1(w) = \|b_01^Tw - \Delta q\|$

$c_2(w) = -w$

$c_3(w) = w-w_{max}$

Ici, on a remplacé dans $c_1$ le vecteur $1_{[n_0,n_f]}$ par le vecteur $1 = 1_{[0,N]} = (1,1,\dots,1)$ car en dehors de cette intervalle $[n_0,n_f]$ la puissance transmise à la voiture est nulle, donc on suppose que $n_0 = 0$ et $n_f = N$ car on pourra en déduire w sur tout intervalle plus large (nulle en dehors).

On remarque que $f$ et $c_2$ sont convexes car linéaires, et que $c_1$ et $c_3$ sont convexes car affines.

On peut montrer l'existence d'une solution si on a $ \Delta q \leq (N+1)b_0w_{max}$. 

En effet dans ce cas on sait que l'ensemble des $w$ tq $c_1(w) = 0, c_2(w) \leq 0$ et $ c_3(w) \leq 0$ est non vide car $w = \frac{\Delta q}{Nb_0}$ y appartient. 

On a de plus que l'ensemble des $w$ tq $c_2(w) \leq 0$ et $ c_3(w) \leq 0$ est un **compact** de $\mathbb{R}^N$ (fermé borné pour la norme 1 en dimension finie). Or comme l'ensemble des $w$ tq $c_1(w) = 0$ est un hyperplan de $\mathbb{R}^N$ espace vectoriel de dimension finie, c'est un **fermé**, et ainsi par intersection d'un compact par un fermé, l'ensemble des $w$ vérifiant les contraintes du problème de minimisation est un **compact non vide**. Ainsi, d'après le théorème de Weierstass (l'hypothèse f continue est vérifiée car f est linéaire), **il existe donc une solution à ce problème**.

Il n'y a pas toujours unicité de la solution. En effet on peut remarquer que si $p = p_0.1$, alors pour tout w vérifiant les contraintes $f(w) = p^Tw = p_0.1^Tw = \frac{\Delta q}{b_0}$, ce qui donne a priori une infinité de solution.

2.Ordre de grandeurs des variables du problème:

$b_0 = \frac{1}{u_0} = \frac{1}{220V} \approx 0.005 V^{-1}$

Le prix du kWh en France est compris entre 13,37 et 17,81 centimes selon EDF, soit $0,1337 \leq p \leq 0,1781$ si on considère que les instants sont toutes les heures.

Pour une borne de recharge de maison, les principaux revendeurs indiquent que $w_{max} \approx 30 kW$.

Pour une seule voiture, nous proposons la méthode intuitive suivante. Tout d'abord il faut vérifier la condition d'existence plus haut. En triant les coordonnées de p, on peut obtenir une permutation $\Phi : [|0,N|] -> [|0,N|]$ telle que $p_{\Phi (1)} \leq p_{\Phi (2)} \leq \dots \leq p_{\Phi (N)}$. En fait cela revient à trier les N instants par coût de l'énergie. Le principe est de charger la voiture au maximum pendant l'instant le moins cher c'est à dire $\Phi (1)$ ici, puis si la voiture n'est toujours pas chargée comme on le souhaite pendant l'instant le moins cher non utilisé ici $\Phi (2)$ etc jusqu'à ce qu'on ait rempli la voiture.

En pseudo-code cela donnerait :

`
DEBUT DE L'ALGORITHME
w <- 0 #vecteur nul
Pour j allant de 0 à N:
    si deltaq - b0*1T*w est supérieure à b0*wmax:
        alors w[phi(j)] <- wmax
    sinon:
        w[phi(j)] <- u0*deltaq - 1T*w
        retourner w
    finsi
finPour
FIN DE L'ALGORITHME
`

Le w obtenu est bien un minimiseur de la fonction coût. De plus il respecte les contraintes car un invariant est ici que toutes les coordonnées de w sont positives et valent au maximum wmax, et à la fin on a bien $b_01^Tw = \Delta q$. Pour la minimisation, on a chargé en priorité la voiture pendant les heures les moins chers.

In [1]:
import numpy as np
from scipy import optimize

In [2]:
def permutation_croissante(p):
    """
    la fonction renvoie un tableau phi qui contient la valeur de phi(i) (phi étant définie dans la question 2) dans la case i
    """
    N = p.size - 1
    phi = np.array([x for x in range(0,N+1)])
    for i in range(0,N):
        pmin = p[phi[i]]
        jmin = i
        for j in range(i,N+1):
            if p[phi[j]] <= pmin:
                pmin = p[phi[j]]
                jmin = j
        phi[jmin], phi[i] = phi[i], phi[jmin]
    return phi

In [3]:
def somme(tab):
    n = tab.size
    somme = 0
    for i in range(0,n):
        somme += tab[i]
    return somme

In [4]:
def minimisation_1voiture(p,deltaq,wmax=30000):
    """
    Renvoie un minimiseur de la fonction coût du problème de recharge pour 1 voiture chargeant de l'instant 0 à l'instant N.
    deltaq est en coulomb (et positif), wmax en W et p en euros.
    """
    N = p.size - 1
    u0 = 220
    assert deltaq <= (N+1)*wmax/u0, 'le temps de recharge sera trop faible pour charger comme demandé la voiture'
    w = np.zeros(N+1)
    phi = permutation_croissante(p)
    for j in range(0, N+1):
        if u0*deltaq - somme(w) >= wmax:
            w[phi[j]] = wmax
        else:
            w[phi[j]] = u0*deltaq - somme(w)
            return(w)

In [5]:
def minimisation_1_voiture(p,deltaq,n0,nf, wmax=30000):
    N = p.size - 1
    assert (0 <= n0) and (n0 <=nf) and (nf<=N), 'il faut que 0<=n0<=nf<=N'
    P = p[n0:nf+1]
    W = minimisation_1voiture(p,deltaq,wmax)
    w = np.concatenate((np.zeros(n0),W,np.zeros(N - nf)))
    return w

In [6]:
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220*2

w1 = minimisation_1_voiture(p,deltaq1,n10,n1f,wmax)
print(w1)

[3.00000000e+04 7.27595761e-12 0.00000000e+00 3.00000000e+04
 0.00000000e+00]


# 3 Etude avancée

3.Pour plusieurs véhicules, on a maintenant le problème suivant:

Posons $w = 
\begin{pmatrix}
w_{1,1} \\
\vdots \\
w_{1,N} \\
w_{2,1} \\
\vdots \\
w_{n_v,N}
\end{pmatrix} \in \mathbb{R}^{n_vN}$

Plus formellement, on a la n-ième coordonnée de $w$ qui est la j-ème coordonnée de $w_i$ avec $(n-1) = (i-1)N + (j-1)$ la division euclidienne de (n-1) par N. En particulier, i et j sont uniques en fonction de n. On les index donc $i_n$ et $j_n$.
Sous cette forme de vecteur de $\mathbb{R}^{n_vN}$, on peut réécrire f de la forme:

$f(w) = \sum_{n=1}^{n_vN} p_{j_n}w_{i_n,j_n} = \sum_{n=1}^{n_vN}f_n(w_{i_n,j_n})$ avec $\forall n, \forall w, f_n(w) = p_{j_n}w$

$$c(w_1, \dots, w_{n_v}) =
\begin{pmatrix}
b_0 1_{[n_{1,0},n_{1,f}]}^T w_1 - \Delta q_1 \\
\Delta q_1 - b_0 1_{[n_{1,0},n_{1,f}]}^T w_1 \\
\vdots \\
b_0 1_{[n_{i,0},n_{i,f}]}^T w_i - \Delta q_i \\
\Delta q_i - b_0 1_{[n_{i,0},n_{i,f}]}^T w_i\\
\vdots \\
b_0 1_{[n_{n_v,0},n_{n_v,f}]}^T w_{n_v} - \Delta q_{n_v} \\
\Delta q_{n_v} - b_0 1_{[n_{n_v,0},n_{n_v,f}]}^T w_{n_v} \\
-w_{1,1} \\
\vdots \\
-w_{i_n,j_n} \\
\vdots \\
-w_{n_v,N} \\
(\sum_{i=1}^{n_v} w_{i,1}) - w_{max} \\
\vdots \\
(\sum_{i=1}^{n_v} w_{i,j}) - w_{max} \\
\vdots \\
(\sum_{i=1}^{n_v} w_{i,N}) - w_{max} \\
\end{pmatrix} \in \mathbb{R}^{2n_v+n_vN + N}
$$

La première contrainte égalité a été réécrite en deux contraintes inégalités. Dans la suite, comme p est à coefficient positif, nous pouvons remarquer que dans les algorithmes de minimisation, seul la contrainte $\Delta q_i - b_0 1_{[n_{i,0},n_{i,f}]}^T w_i$ pourra être retenu car comme on cherche à minimiser le prix, on cherchera à minimiser chaque $w_i$ et donc l'autre contrainte sera casiment respectée.

Pour séparer les contraintes, on pose 

$$\forall n \in [|1,n_vN|], c_n(w_{i_n,j_n}) =
\begin{pmatrix}
0 \\
\vdots \\
0\\
b_0 \mathbb{1}_{[n_{i_n,0},n_{i_n,f}]}(j_n) w_{i_n,j_n} - \frac{\Delta q_{i_n}}{N} \\
\frac{\Delta q_{i_n}}{N} - b_0 \mathbb{1}_{[n_{i_n,0},n_{i_n,f}]}(j_n) w_{i_n,j_n} \\
0\\
\vdots \\
0\\
-w_{i_n,j_n} \\
0\\
\vdots \\
0\\
w_{i_n,j_n} - \frac{w_{max}}{n_v} \\
0\\
\vdots \\
0\\
\end{pmatrix} \in \mathbb{R}^{2n_v+n_vN + N}
$$

Précisons que les coordonnées non nulles sont celles en positions $2(i_n-1) + 1$,  $2i_n$,  $2n_v +n$  et $2n_v + n_vN + j_n$. On obtient bien avec ces définitions $c(w) = \sum_{n=1}^{n_vN} c_n(w_{i_n,j_n})$

**Il apparaît donc possible de réaliser un algorithme de décomposition/coordination par méthode d'Uzawa.**

4.Un peu de notations:

$w = 
\begin{pmatrix}
w_{1,1} \\
\vdots \\
w_{1,N} \\
w_{2,1} \\
\vdots \\
w_{2,N}
\end{pmatrix} \in \mathbb{R}^{2N}$

Attention, dans la suite nous noterons $w_n$ pour $w_{i_n,j_n}$

$f(w) = f_1(w_1) + f_{2N}(w_{2N})$ avec $\forall n, \forall w \in \mathbb{R}^{N}, f_n(w) = p_{j_n}w = \left\{
    \begin{array}{ll}
        p_nw & \mbox{si } n \leq N \\
        p_{n-N}w & \mbox{sinon.}
    \end{array}
\right.$

Dans le cas $n \leq N \mbox{, i.e } i_n = 1 \mbox{ , } j_n = n$

$c_n(w_n) =
\begin{pmatrix}
b_0 \mathbb{1}_{[n_{1,0},n_{1,f}]}(n) w_n - \frac{\Delta q_{1}}{N} \\
\frac{\Delta q_{1}}{N} - b_0 \mathbb{1}_{[n_{1,0},n_{1,f}]}(j) w_n \\
0\\
\vdots \\
0\\
-w_n \\
0\\
\vdots \\
0\\
w_n - \frac{w_{max}}{n_v} \\
0\\
\vdots \\
0\\
\end{pmatrix} \in \mathbb{R}^{4 + 3N}$

Dans le cas $n > N \mbox{, i.e } i_n = 2 \mbox{ , } j_n = n - N$

$c_n(w_n) =
\begin{pmatrix}
0\\
0\\
b_0 \mathbb{1}_{[n_{2,0},n_{2,f}]}(n-N) w_n - \frac{\Delta q_{2}}{N} \\
\frac{\Delta q_{2}}{N} - b_0 \mathbb{1}_{[n_{2,0},n_{2,f}]}(n-N) w_n \\
0\\
\vdots \\
0\\
-w_n \\
0\\
\vdots \\
0\\
w_n - \frac{w_{max}}{n_v} \\
0\\
\vdots \\
0\\
\end{pmatrix} \in \mathbb{R}^{4 + 3N}$

Nous venons de nous rendre compte que nous avons changer de notation dans la partie 3. Nous avons considérer que N est le nombre d'instant et non plus N+1. Nous garderons cette convention dans l'algorithme pour suivre les écritures mathématiques ci dessus.

In [11]:
def uzawa_fixed_step(fun, grad_fun, c, grad_c, x0, l, rho, lambda0 = 1.0, max_iter = 10000, epsilon_grad_L = 1e-8):
    k = 0
    xk = x0
    lambdak = lambda0
    grad_Lagrangienk_xk = grad_fun(xk) + np.dot(lambdak,grad_c(xk))
    while ((k<max_iter) and (np.linalg.norm(grad_Lagrangienk_xk)>epsilon_grad_L)):
        grad_Lagrangienk_xk = grad_fun(xk) + np.dot(lambdak,grad_c(xk))
        pk = -grad_Lagrangienk_xk
        xk = xk + l*pk;        
        lambdak = np.maximum(0, lambdak + rho*c(xk))
        k = k + 1
    return xk

In [72]:
u0 = 220
def deffn(p,n,N):
    if n<N:
        return lambda wn: p[n]*wn
    else:
        return lambda wn: p[n-N]*wn

def defgradfn(p,n,N):
    if n<N:
        return lambda wn: p[n]
    else:
        return lambda wn: p[n-N]

def defcn(p,n,N,indicatrice1,indicatrice2):
    def cn(wn):
        c = np.zeros(4+3*N)
        if n<N:
            c[0] = wn*indicatrice1[n] - u0*deltaq1/N
            c[1] = -c[0]
            c[0] = 0 #contrainte superflue
            c[4+2*N+n] = wn - wmax/2
        else:
            c[2] = indicatrice2[n-N]*wn - u0*deltaq2/N
            c[3] = -c[2]
            c[2] = 0 #contrainte superflue
            c[4+N+n] = wn - wmax/2
        c[4+n] = - wn
        return c
    return cn

def defc(p,N,indicatrice1,indicatrice2):
    def c(wk):
        c0 =np.zeros(4+3*N)
        for n in range(0,2*N):
            cn = defcn(p,n,N,indicatrice1,indicatrice2)
            c0 = c0 + cn(wk[n])
        return c0
    return c



def defgradcn(p,n,N,indicatrice1,indicatrice2):
    gradcn = np.zeros(4+3*N)
    if n<N:
        gradcn[0] = indicatrice1[n]
        gradcn[1] = -gradcn[0]
        gradcn[0] = 0 #contrainte superflue
        gradcn[4+2*N+n] = 1
    else:
        gradcn[2] = indicatrice2[n-N]
        gradcn[3] = -gradcn[2]
        gradcn[2] = 0 #contrainte superflue
        gradcn[4+N+n] = 1
    gradcn[4+n] = - 1
    return lambda wn: gradcn

def defgradc(p,N,indicatrice1,indicatrice2):
    def gradc(wk):
        gradc0 =np.zeros(4+3*N)
        for n in range(0,2*N):
            gradcn = defgradcn(p,n,N,indicatrice1,indicatrice2)
            gradc0 = gradc0 + gradcn(wk[n])
        return gradc0
    return gradc

def gn(p,n,N,lambdak,indicatrice1,indicatrice2):
    fn = deffn(p,n,N)
    cn = defcn(p,n,N,indicatrice1,indicatrice2)
    def g(wn):
        return fn(wn) + np.dot(cn(wn),np.transpose(lambdak))
    return g
    
def gradgn(p,n,N,lambdak,indicatrice1,indicatrice2):
    gradfn = defgradfn(p,n,N)
    gradcn = defgradcn(p,n,N,indicatrice1,indicatrice2)
    def gradg(wn):
        return gradfn(wn) + np.dot(gradcn(wn),np.transpose(lambdak))
    return gradg

In [73]:
def decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, rho, eps = 10**(-8),maxiter = 100):
    """
    """
    N = p.size
    u0 = 220
    lambdak = np.zeros(4+3*N)
    lambdak1 = np.ones(4+3*N)
    
    indicatrice1 = np.zeros(N)
    for n in range(n10,n1f+1):
        indicatrice1[n] = 1
    
    indicatrice2 = np.zeros(N)
    for n in range(n20,n2f+1):
        indicatrice2[n] = 1
    
    Abs = np.vectorize(lambda x: max(0,x)) #une projection sur R+^m
    
    w10 = minimisation_1_voiture(p,deltaq1,n10,n1f,wmax/2) #wmax/2 pour avoir somme des wi <= wmax
    w20 = minimisation_1_voiture(p,deltaq2,n20,n2f,wmax/2)
    w0 = np.concatenate((w10,w20)) #pour commencer les itérations, on prend la concatenation des solutions individuelles
    wk = w0
    c = defc(p,N,indicatrice1,indicatrice2)
    k = 0
    while((k<maxiter) and (np.linalg.norm(lambdak1-lambdak)>eps)):
        lambdak = lambdak1
        for n in range(0,2*N):
            g = gn(p,n,N,lambdak,indicatrice1,indicatrice2)
            gradg = gradgn(p,n,N,lambdak,indicatrice1,indicatrice2)
            cn = defcn(p,n,N,indicatrice1,indicatrice2)
            gradcn = defgradcn(p,n,N,indicatrice1,indicatrice2)
            wk[n] = uzawa_fixed_step(g, gradg, cn, gradcn, wk[n], rho, rho,np.ones(4+3*N))
        lambdak1 = Abs(lambdak + rho*(c(wk)))
        k +=1
    print(k)
    return wk[0:N]*indicatrice1,wk[N:2*N]*indicatrice2


In [110]:
#essai sans la contrainte superflue
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220
n20, n2f = 1, 3
deltaq2 =0
w1, w2 = decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, 0.01, eps = 10**(-8))
print(w1)
print(w2)

4
[14958.94310569 14945.62837651 14945.62837651     0.        ]
[ 0.         46.61400174 46.61400174 46.61400174]


In [111]:
(somme(w1) - 220*deltaq1)/220/deltaq1

0.4950066619568755

In [113]:
#essai sans la contrainte superflue
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220/3
n20, n2f = 1, 3
deltaq2 =wmax/220/3
w1, w2 = decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, 0.01, eps = 10**(-8))
print(w1)
print(w2)

4
[10046.56000001  5026.10775299  5026.10775299     0.        ]
[    0.          5026.10775299 10028.79999997  5026.10775299]


In [114]:
(somme(w1) - 220*deltaq1)/deltaq1/220,(somme(w2) - 220 * deltaq2)/deltaq2/220

(1.0098775505992086, 1.0081015505945976)

In [115]:
#essai sans la contrainte "superflue"
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220
n20, n2f = 1, 2
deltaq2 =wmax/220
w1, w2 = decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, 0.1, eps = 10**(-8))
print(w1)
print(w2)

70
[13779.26615628 13374.57163428 13375.58160277     0.        ]
[    0.         15000.00000004 15000.00000004     0.        ]


In [116]:
(somme(w1) - 220*deltaq1)/deltaq1/220,(somme(w2) - 220 * deltaq2)/deltaq2/220

(0.3509806464443275, 2.470915205776691e-12)

On a des approximations "hautes" de la fonction de puissance retenue pour la charge. Le point positif est qu'on aura donc des voitures au moins chargées comme demandé, mais ce sera à un coût trop cher. Essayons une dernière fois en prenant en compte la contrainte que nous avons jugé "superflue" en la modifiant très légèrement à cause de l'arrondi.

In [117]:
eps = 10**(-4)
u0 = 220
def deffn(p,n,N):
    if n<N:
        return lambda wn: p[n]*wn
    else:
        return lambda wn: p[n-N]*wn

def defgradfn(p,n,N):
    if n<N:
        return lambda wn: p[n]
    else:
        return lambda wn: p[n-N]

def defcn(p,n,N,indicatrice1,indicatrice2):
    def cn(wn):
        c = np.zeros(4+3*N)
        if n<N:
            c[0] = wn*indicatrice1[n] - u0*deltaq1/N*(1+eps) #eps pour éviter les erreurs d'arrondis
            c[1] = -c[0]
            #c[0] = 0 #contrainte superflue
            c[4+2*N+n] = wn - wmax/2
        else:
            c[2] = indicatrice2[n-N]*wn - u0*deltaq2/N*(1+eps) #eps pour éviter les erreurs d'arrondis
            c[3] = -c[2]
            #c[2] = 0 #contrainte superflue
            c[4+N+n] = wn - wmax/2
        c[4+n] = - wn
        return c
    return cn

def defc(p,N,indicatrice1,indicatrice2):
    def c(wk):
        c0 =np.zeros(4+3*N)
        for n in range(0,2*N):
            cn = defcn(p,n,N,indicatrice1,indicatrice2)
            c0 = c0 + cn(wk[n])
        return c0
    return c



def defgradcn(p,n,N,indicatrice1,indicatrice2):
    gradcn = np.zeros(4+3*N)
    if n<N:
        gradcn[0] = indicatrice1[n]
        gradcn[1] = -gradcn[0]
        #gradcn[0] = 0 #contrainte superflue
        gradcn[4+2*N+n] = 1
    else:
        gradcn[2] = indicatrice2[n-N]
        gradcn[3] = -gradcn[2]
        #gradcn[2] = 0 #contrainte superflue
        gradcn[4+N+n] = 1
    gradcn[4+n] = - 1
    return lambda wn: gradcn

def defgradc(p,N,indicatrice1,indicatrice2):
    def gradc(wk):
        gradc0 =np.zeros(4+3*N)
        for n in range(0,2*N):
            gradcn = defgradcn(p,n,N,indicatrice1,indicatrice2)
            gradc0 = gradc0 + gradcn(wk[n])
        return gradc0
    return gradc

def gn(p,n,N,lambdak,indicatrice1,indicatrice2):
    fn = deffn(p,n,N)
    cn = defcn(p,n,N,indicatrice1,indicatrice2)
    def g(wn):
        return fn(wn) + np.dot(cn(wn),np.transpose(lambdak))
    return g
    
def gradgn(p,n,N,lambdak,indicatrice1,indicatrice2):
    gradfn = defgradfn(p,n,N)
    gradcn = defgradcn(p,n,N,indicatrice1,indicatrice2)
    def gradg(wn):
        return gradfn(wn) + np.dot(gradcn(wn),np.transpose(lambdak))
    return gradg

In [118]:
def decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, rho, eps = 10**(-8),maxiter = 100):
    """
    """
    N = p.size
    u0 = 220
    lambdak = np.zeros(4+3*N)
    lambdak1 = np.ones(4+3*N)
    
    indicatrice1 = np.zeros(N)
    for n in range(n10,n1f+1):
        indicatrice1[n] = 1
    
    indicatrice2 = np.zeros(N)
    for n in range(n20,n2f+1):
        indicatrice2[n] = 1
    
    Abs = np.vectorize(lambda x: max(0,x)) #une projection sur R+^m
    
    w10 = minimisation_1_voiture(p,deltaq1,n10,n1f,wmax/2) #wmax/2 pour avoir somme des wi <= wmax
    w20 = minimisation_1_voiture(p,deltaq2,n20,n2f,wmax/2)
    w0 = np.concatenate((w10,w20)) #pour commencer les itérations, on prend la concatenation des solutions individuelles
    wk = w0
    c = defc(p,N,indicatrice1,indicatrice2)
    k = 0
    while((k<maxiter) and (np.linalg.norm(lambdak1-lambdak)>eps)):
        lambdak = lambdak1
        for n in range(0,2*N):
            g = gn(p,n,N,lambdak,indicatrice1,indicatrice2)
            gradg = gradgn(p,n,N,lambdak,indicatrice1,indicatrice2)
            cn = defcn(p,n,N,indicatrice1,indicatrice2)
            gradcn = defgradcn(p,n,N,indicatrice1,indicatrice2)
            wk[n] = uzawa_fixed_step(g, gradg, cn, gradcn, wk[n], rho, rho,np.ones(4+3*N))
        lambdak1 = Abs(lambdak + rho*(c(wk)))
        k +=1
    print(k)
    return wk[0:N]*indicatrice1,wk[N:2*N]*indicatrice2


In [119]:
#essai avec la contrainte superflue
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220
n20, n2f = 1, 3
deltaq2 =0
w1, w2 = decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, 0.01, eps = 10**(-8))
print(w1)
print(w2)

100
[9361.21989388 9361.19927123 9361.19927123    0.        ]
[ 0.         -0.09594421 -0.09594421 -0.09594421]


In [120]:
(somme(w1) - 220*deltaq1)/220/deltaq1

-0.06387938545565

On est très proche et on remarque qu'on s'est arrêté à cause du nombre maximal d'itérations imposé. Réessayons avec les exemples précédents pour voir si nous pouvons conjecturer une convergence de l'algorithme.

In [122]:
#essai avec la contrainte superflue
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220/3
n20, n2f = 1, 3
deltaq2 =wmax/220/3
w1, w2 = decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, 0.01, eps = 10**(-8))
print(w1)
print(w2)

100
[3112.43434016 3112.41371751 3112.41371751    0.        ]
[   0.         3112.41371751 3112.41371751 3112.41371751]


In [123]:
(somme(w1) - 220*deltaq1)/deltaq1/220,(somme(w2) - 220 * deltaq2)/deltaq2/220

(-0.06627382248210542, -0.06627588474751737)

In [125]:
#essai avec la contrainte "superflue"
phc, php = 0.1337, 0.1781

p = np.array([phc,php,php,php])
wmax = 30000
n10, n1f = 0, 2
deltaq1 = wmax/220
n20, n2f = 1, 2
deltaq2 =wmax/220
w1, w2 = decomposition_coordination(p,n10,n1f,n20,n2f,deltaq1,deltaq2, wmax, 0.1, eps = 10**(-8))
print(w1)
print(w2)

100
[10001.03093184 10000.98453407 10000.98453407    -0.        ]
[   0.         5136.00844951 5136.00844951   -0.        ]


In [126]:
(somme(w1) - 220*deltaq1)/deltaq1/220,(somme(w2) - 220 * deltaq2)/deltaq2/220

(9.999999901325888e-05, -0.6575994366991066)

## CONCLUSION
Nous n'aurons pas le temps d'aller plus loin. Quelques petits points pour expliquer les résultats obtenus. On est plutôt content à la fin puisque les solutions obtenus vérifient bien la première contrainte sur la charge des voitures. Cependant, il n'y a plus comme dans les 3 exemples qui précédaient une recharge principalement sur l'heure numéro 0 (une heure creuse) pour la voiture 1 ce qui est étonnant. On peut même voir cela pour toutes les voitures que la recharge est uniforme par rapport au temps. Ce n'est pas du tout l'effet voulu de l'algorithme. On peut supposer que c'est parce qu'on a donné beaucoup d'importance à la contrainte 1 en réimposant une contrainte qui nous paraissait superflue.